In [ ]:
import cv2
import os
import glob
import time
from PIL import Image
import numpy as np
import PIL
import imageio
import random
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from UtilityTest import DepthDataset
from UtilityTest import ToTensor

In [ ]:
Location_video='/workspace/test_vid/VID_20200102_142929.mp4'

In [ ]:
#Extract the video from zipfile
from zipfile import ZipFile
zf = ZipFile('/workspace/test_vid.zip', 'r')
zf.extractall('/workspace/')
zf.close()

In [ ]:
#converting video into frames
vidcap = cv2.VideoCapture(Location_video)
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("/workspace/test_vid/Frames/frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

In [ ]:
#location of video frames
loc_img='/workspace/test_vid/Frames/'

#ploting few sample frames
depth_dataset = DepthDataset(root_dir=loc_img)
fig = plt.figure()
len(depth_dataset)
for i in range(len(depth_dataset)):
    sample = depth_dataset[i]

    print(i, sample['image'].size)


    plt.imshow(sample['image'])
    plt.figure()


    if i == 2:
        plt.show()
        break

In [ ]:
depth_dataset = DepthDataset(root_dir=loc_img,transform=transforms.Compose([ToTensor()]))
# depth_dataset
batch_size=1
train_loader=torch.utils.data.DataLoader(depth_dataset, batch_size)
# train_loader
dataiter = iter(train_loader)
images = dataiter.next()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils as utils
import torchvision.utils as vutils    
import torchvision.models as models
from Mobile_model import Model
model = Model().cuda()
model = nn.DataParallel(model)
#loading the the trained model
model.load_state_dict(torch.load('/workspace/17.pth'))
model.eval()


In [ ]:
os.mkdir('/workspace/test_vid/depth_frames/')

#generating the depth image of frames

import matplotlib.cm as cm
for i,sample_batched1  in enumerate (train_loader):
    image1 = torch.autograd.Variable(sample_batched1['image'].cuda())
    
    outtt=model(image1 )
    x=outtt.detach().cpu().numpy()
    x.shape
    x=x.reshape(240,320)
    img=x
    scale_percent = 200 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    plt.imsave('/workspace/test_vid/depth_frames/geeks%d.jpg' %i, resized, cmap='inferno')


In [ ]:
from os.path import isfile, join
 
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
 

In [ ]:
    #convert frames to videos
    pathIn= '/workspace/test_vid/depth_frames/'
    pathOut = '/workspace/test_vid/video1.mp4'
    #enter the correct frame rates
    fps = 30.01
    convert_frames_to_video(pathIn, pathOut, fps)

In [ ]:
#converting the frames to gif

filenames='/workspace/test_vid/depth_frames/'
files = [f for f in os.listdir(filenames) if isfile(join(filenames, f))]

files.sort(key = lambda x: int(float(x[5:-4])))
files
images = []
for i,filename in enumerate(files):
    if i%10==0:
        images.append(imageio.imread(os.path.join('/workspace/test_vid/depth_frames/',filename)))
        print(filename)
imageio.mimsave('/workspace/test_vid/movie_depth.gif', images)

In [ ]:
from skimage import transform,io
filenames='/workspace/test_vid/Frames/'
files = [f for f in os.listdir(filenames) if isfile(join(filenames, f))]

files.sort(key = lambda x: int(float(x[5:-4])))
files
images = []
for i,filename in enumerate(files):
    if i%10==0:
       

            # resize to 28x28
        grey=(imageio.imread(os.path.join('/workspace/test_vid/Frames/',filename)))
        image = transform.resize(grey, (480,640), mode='symmetric', preserve_range=True)
        images.append(image)
        print(filename)
imageio.mimsave('/workspace/test_vid/movie_real.gif', images)